# DSECS: Dipolar Spherical Elementary Current Systems (WIP)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import datetime as dt
from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import dsecs
import matplotlib.pyplot as plt

In [ ]:
# Enable logging in notebook. Remove this line to disable.
logging.basicConfig(level=logging.INFO, force=True)

## Fetching inputs to the toolbox

In [ ]:
def data_params(spacecraft="A"):
    return dict(
        collection=f"SW_OPER_MAG{spacecraft}_LR_1B",
        measurements=["B_NEC"],
        models=["Model = CHAOS"],  # currently must use name "Model"
        auxiliaries=["QDLat"],
        start_time="2016-03-18T11:00:00",
        end_time="2016-03-18T11:50:00",
        server_url="https://vires.services/ows",
        options=dict(asynchronous=False, show_progress=False),
    )


data = create_paldata(
    PalDataItem.from_vires(**data_params("A")),
    PalDataItem.from_vires(**data_params("C")),
)

print(data)

## Applying the DSECS process

### Preprocess

This initial process adds in Apex coordinates to the datasets.

In [ ]:
p1 = dsecs.processes.Preprocess()
p1(data)
print(data)

### Analysis

This process performs the DSECS analysis. It currently takes about 3 minutes to process one pass over the mid-latitudes.

In [ ]:
%%time
p2 = dsecs.processes.Analysis()
data = p2(data)

Outputs are stored under a new "DSECS_output" branch of the datatree. This branch is further divided into N branches ("0", "1", "2", ...) depending on how many passes have been analysed, and under each branch into "currents" (the estimated current densities), "Fit_Alpha" and "Fit_Charlie" (the residuals for the fitted magnetic data).

In [ ]:
print(data["DSECS_output"])

## Plotting the outputs and fitted magnetic field 

### Simple line plots of the currents from the central latitudinal slice

The outputs in the datatree are enumerated, for each analyzed equatorial crossing included inside the timestamps. ("DSECS_output/0/currents","DSECS_output/1/currents" etc.) 

In [ ]:
print(data["DSECS_output/0/currents"].coords)
print(data["DSECS_output/0/currents"].data_vars)

In [ ]:
latitudes = data["DSECS_output/0/currents"]["Latitude"][:, 3]
fig, ax = plt.subplots(4, 1, figsize=(10, 20))
lineE = ax[0].plot(
    latitudes,
    data["DSECS_output/0/currents"]["JEastTotal"][:, 3],
    "r",
    label="Eastward",
)
lineN = ax[0].plot(
    latitudes,
    data["DSECS_output/0/currents"]["JNorthTotal"][:, 3],
    "k",
    label="Northward",
)
ax[0].set_title("Total Current")
ax[0].set_ylabel("Current density (A/km)")


lineE = ax[1].plot(
    latitudes, data["DSECS_output/0/currents"]["JEastCf"][:, 3], "r", label="Eastward"
)
lineN = ax[1].plot(
    latitudes, data["DSECS_output/0/currents"]["JNorthCf"][:, 3], "k", label="Northward"
)
ax[1].set_title("Curl free current")
ax[1].set_ylabel("Current density (A/km)")

# ax[0].set_title('Total Current (DF + CF)')

lineE = ax[2].plot(
    latitudes, data["DSECS_output/0/currents"]["JEastDf"][:, 3], "r", label="Eastward"
)
lineN = ax[2].plot(
    latitudes, data["DSECS_output/0/currents"]["JNorthDf"][:, 3], "k", label="Northward"
)
ax[2].set_title("Divergence free current")
ax[2].set_ylabel("Current density (A/km)")

liner = ax[3].plot(
    latitudes,
    data["DSECS_output/0/currents"]["Jr"][:, 3],
    "r",
    label="Radial current density",
)
ax[3].set_title("Radial current density")
ax[3].set_ylabel("Current density (nA/m^2)")

for axv in ax:
    axv.set_xlabel("Latitude", loc="left")
    axv.legend()